In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

gpus

2025-05-28 14:30:41.016749: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-28 14:30:41.116434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748457041.155285    5664 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748457041.166686    5664 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748457041.248242    5664 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
import evaluate
import json

# === Load BLEURT metric with the smallest model ===
bleurt = evaluate.load("bleurt", config_name="bleurt-base-512")


INFO:tensorflow:Reading checkpoint /home/j/.cache/huggingface/metrics/bleurt/bleurt-base-512/downloads/extracted/3dac74417ac597a79d1308cb03cd22526d55c1a52b297c2f311c31cb67224477/bleurt-base-512.


INFO:tensorflow:Reading checkpoint /home/j/.cache/huggingface/metrics/bleurt/bleurt-base-512/downloads/extracted/3dac74417ac597a79d1308cb03cd22526d55c1a52b297c2f311c31cb67224477/bleurt-base-512.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint bert_custom


...

INFO:tensorflow:Creating Eager Mode predictor.


INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [3]:
from datasets import load_dataset

ds = load_dataset('billingsmoore/temp', split='train')

references = ds['en']
predictions = ds['small_predictions']

In [8]:
# === Compute BLEURT scores ===
results = bleurt.compute(predictions=predictions, references=references)

In [10]:
results

{'scores': [-0.15208300948143005,
  0.9593027830123901,
  0.6225766539573669,
  -0.2820627689361572,
  -0.1863727867603302,
  -0.43291139602661133,
  -0.5732894539833069,
  0.1545206606388092,
  0.10523593425750732,
  -0.8846349120140076,
  0.038565412163734436,
  -1.2352956533432007,
  -0.7578621506690979,
  -0.24161233007907867,
  -0.4433731436729431,
  -0.44201409816741943,
  0.3366703391075134,
  -0.8576117753982544,
  -0.20685122907161713,
  -0.3636091351509094,
  -0.43216976523399353,
  -0.1443851888179779,
  -0.24281570315361023,
  -0.01237514615058899,
  -0.8278073072433472,
  0.06733671575784683,
  0.5572270154953003,
  -0.026405319571495056,
  0.047230541706085205,
  -0.7240908145904541,
  -0.5124080777168274,
  -0.3725215494632721,
  -0.5027583241462708,
  -0.19704526662826538,
  -0.6791120767593384,
  -0.27711021900177,
  0.015713930130004883,
  0.03213576227426529,
  -0.6595355272293091,
  -0.34306538105010986,
  0.03244782239198685,
  1.011069655418396,
  0.38734376430511

In [9]:


# === Extract scores and compute mean ===
bleurt_scores = results["scores"]
mean_bleurt = sum(bleurt_scores) / len(bleurt_scores)

# === Print and save results ===
print(f"BLEURT mean score: {mean_bleurt:.4f}")

output = {
    "bleurt_model": "bleurt-base-512",
    "mean_score": mean_bleurt,
    "scores": bleurt_scores,
    "num_samples": len(bleurt_scores),
}

with open("small_bleurt_base512_results.json", "w") as f:
    json.dump(output, f, indent=2)


BLEURT mean score: -0.1903
